<a href="https://colab.research.google.com/github/talalba/Instagram_Like_Predictor/blob/master/lid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [224]:
# import the necessary packages
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
import importlib
import itertools
import nltk
from nltk import ngrams
from nltk import word_tokenize
import string
import collections
import pandas as pd
import numpy as np
import re
from random import sample 

from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [323]:
# load the data
directoryPath = '/content/drive/My Drive/lid/'
# df = pd.read_csv(directoryPath+'formatted_data.csv',sep=';',encoding='latin1')
df = pd.read_csv(directoryPath+'formatted_data.csv',sep=';')
df.shape

# filter the data to remove unwanted characters
# function to remove punctuation 
def remove_punctuation(text): 
    translator = str.maketrans('', '', string.punctuation) 
    return text.translate(translator) 

# do we keep whitespace?
# function to remove whitespace 
def remove_whitespace(text): 
    return  " ".join(text.split()) 

docs = df['text'].str.replace('\d+', '') # remove numbers in strings
docs = docs.str.lower() # make letters lower case
docs = [remove_whitespace(text) for text in docs]

# get sentences from paragraphs
docs = [sent_tokenize(text) for text in docs]


(21, 3)

In [481]:
# split data into train and test set
# number of training sentences
no_train_sentences = 200
no_test_sentences = 1
split_data = lambda x:  sample(x,no_train_sentences + no_test_sentences)

z = list(map(split_data, docs))

# index the train and test data from the list z
train_data = [item[:no_train_sentences] for item in z]
test_data = [item[-no_test_sentences:] for item in z]

# combine the list of senetnces to a list of strings 
train_data = [''.join(x) for x in train_data]
test_data = [''.join(x) for x in test_data]

# remove punctuation from the string
train_data2 = [remove_punctuation(text) for text in train_data]
test_data2 = [remove_punctuation(text) for text in test_data]

D = get_language_id(1,train_data2,test_data2)
D.idxmin(axis=1).to_string(index=False)


0     bg
1     sk
2     da
3     de
4     el
5     en
6     es
7     et
8     fi
9     fr
10    hu
11    it
12    lt
13    en
14    en
15    pl
16    pt
17    ro
18    sk
19    sl
20    sv
dtype: object

In [0]:
# to test one sentence or more
test_sentence = pd.DataFrame(['I am going to play the game and conquer the world'],columns = ['text'])

docs = test_sentence['text'].str.replace('\d+', '') # remove numbers in strings
docs = docs.str.lower() # make letters lower case
test_data = [remove_whitespace(text) for text in docs]
test_data = [''.join(x) for x in test_data]


len(test_data2)
# remove punctuation from the string
test_data2 = [remove_punctuation(text) for text in test_data]
distance_mat = get_language_id(1,train_data2,test_data2)


test_data2 = test_data
train_data2 = train_data2

D.idxmin(axis=1).to_string(index=False)


In [0]:
# this function performs language identification given train and test data and the ngram number 
# train_data, test_data contain a list for each language

def get_language_id(ngram_number,train_data,test_data):
    
  # perform statistics on training data for all languages
  # define function to get unigram frequencies 
  get_frequency_unigram = lambda x:  nltk.FreqDist((ngrams(' '.join(map(str, x)), ngram_number)))

  # get unigram counts for train and test data
  list_unigram_train = list(map(get_frequency_unigram, train_data))
  list_unigram_test = list(map(get_frequency_unigram, test_data))

  # make dataframe
  train_data_df = pd.DataFrame(list_unigram_train,index=df['language'])
  test_data_df = pd.DataFrame(list_unigram_test,index=df['language'])

  # a count of 0.5 is added to events seen in at least
  # one language in the training set
  np.max(train_data_df.isna().sum())

  # all colums have atleast one nonempty values given how python constructs dictionaries
  # thus we replace all NAs with .5
  train_data_df = train_data_df.fillna(.5)

  # get probabilities from counts by dividing the sum
  train_data_df = train_data_df.div(train_data_df.sum(axis=1), axis=0)
  test_data_df = test_data_df.div(test_data_df.sum(axis=1), axis=0)

  # get common ngram columns between train and test data
  common_cols =  np.intersect1d(train_data_df.columns, test_data_df.columns)
  len(common_cols)

  train_data_df2 = train_data_df[common_cols]
  test_data_df2 = test_data_df[common_cols]

  # D = pd.DataFrame(index=train_data_df.index, columns=train_data_df.index)

  D = pd.DataFrame(columns=train_data_df.index)
  


  # perform language indetification using relative entropy
  for ii in range(0,len(test_data)):

    # compute the KL distance 
    # D = (test_data_df2.iloc[ii,:]*np.log(test_data_df2.iloc[ii,:].div(train_data_df2))).sum(axis=1)
    # D.iloc[ii,:] = (test_data_df2.iloc[ii,:]*np.log(test_data_df2.iloc[ii,:].div(train_data_df2))).sum(axis=1)
    d = (test_data_df2.iloc[ii,:]*np.log(test_data_df2.iloc[ii,:].div(train_data_df2))).sum(axis=1)

    D = D.append(d, ignore_index=True)


    # find index of the train language which has the minimum score (closest language in the train set)
    # index = np.argmin(D.iloc[ii,:])
    
    # print(index)

  return D

In [0]:
# Highlighting the limits of your model

# no spatial order
# size of the feature space increase as bigger n grams are used
# charcter level doesnot capture word level information 

In [0]:
# # prepare the train and test data

# # split data into train and test set
# test_data_size = .8 # use 20% data for testing, 80% for training

# # get all the words for each language and save it in a list
# word_list = [i.split() for i in  docs]

# # generate training and test data for each language using a map fucntion
# # we will split the words in each language into train and test
# # this ignores the temporal order of the words
# split_data = lambda x:  train_test_split(x,test_size=test_data_size)

# # get a list which contains the test and train data splits
# z = list(map(split_data, word_list))
# # z2 = [train_test_split(x,test_size=.2,radnom_state=42) for x in word_list];

# # index the train and test data from the list z
# train_data = [item[0] for item in z]
# test_data = [item[1] for item in z]